Use virtual environment: ors_env

# <font color = 'purple'>Quantifying Accessiblity of Amenities
    
- In this notebook, we will quantify the accessibility of different types of amenities from each apartment
- In some cases, measuring the distance to the amenity of interest is appropriate (e.g. distance to the Central Business District, distance to nearest train station)
- In other cases, the quantity of amenities is pertinent. For example, a larger number of F&B establishments, or a larger area of parks nearby is of greater value
- In other cases, quality is also a consideration. Children residing within 2km of a Primary school receive priority for registration, and so the quality/ ranking of schools within 2km of an apartment would be of interest to parents

In [25]:
import folium
import geopandas as gpd
import time
import pandas as pd
import numpy as np
import math
import requests
import copy
import json
from shapely.geometry import Point, LineString, Polygon
import shapely.speedups
shapely.speedups.enable()

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from tqdm import tqdm
tqdm.pandas()

## <font color = 'blue'> Create 400m and 2km buffers around each block

### Import and convert resi blocks to GeoDataframe

In [2]:
fp_resiblks = r"Clean Datasets\resi_blks2.csv"
resi_blks = pd.read_csv(fp_resiblks,usecols=['blk_no','street','Latitude','Longitude','Postcode'])

resi_blks.head()

,blk_no,street,Latitude,Longitude,Postcode
0,1,BEACH RD,1.303671,103.864479,190001
1,1,BEDOK STH AVE 1,1.320852,103.933721,460001
2,1,CHAI CHEE RD,1.327969,103.922716,461001
3,1,CHANGI VILLAGE RD,1.388610,103.988093,500001
4,1,DELTA AVE,1.292075,103.828584,160001


In [3]:
#check that each row is distinct (i.e. a different block)
resi_blks[resi_blks.duplicated(subset=['Latitude','Longitude','Postcode'],keep=False)]

,blk_no,street,Latitude,Longitude,Postcode


In [4]:
#ensure no nulls
resi_blks.isnull().values.any()

False

In [5]:
# rep each blk as shapely point
resi_blks2 = resi_blks.copy()
resi_blks2['geometry'] = resi_blks2.apply(lambda row: Point(row.Longitude, row.Latitude), axis=1)

resi_blks2.head()

,blk_no,street,Latitude,Longitude,Postcode,geometry
0,1,BEACH RD,1.303671,103.864479,190001,POINT (103.864478660925 1.3036713506088)
1,1,BEDOK STH AVE 1,1.320852,103.933721,460001,POINT (103.933721091441 1.32085208689731)
2,1,CHAI CHEE RD,1.327969,103.922716,461001,POINT (103.922716018139 1.32796879176302)
3,1,CHANGI VILLAGE RD,1.388610,103.988093,500001,POINT (103.988093482829 1.3886100383707)
4,1,DELTA AVE,1.292075,103.828584,160001,POINT (103.828584077626 1.2920752508431)


In [6]:
#convert to geodataframe
resi_blks2 = gpd.GeoDataFrame(resi_blks2,crs="EPSG:4326")

In [7]:
resi_blks2.isnull().values.any()

False

### Create Buffers

In [8]:
# reproject to EPSG 32648 as we need to take distance measurement.
resi_blks2 = resi_blks2.to_crs("EPSG:32648")
resi_blks2.crs

<Projected CRS: EPSG:32648>
Name: WGS 84 / UTM zone 48N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 102°E and 108°E, northern hemisphere between equator and 84°N, onshore and offshore. Cambodia. China. Indonesia. Laos. Malaysia - West Malaysia. Mongolia. Russian Federation. Singapore. Thailand. Vietnam.
- bounds: (102.0, 0.0, 108.0, 84.0)
Coordinate Operation:
- name: UTM zone 48N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [30]:
buffer_distances = [400,800,1200,1600,2000] #change this to adjust buffers

resi_blks3 = resi_blks2.copy()

for i in range(len(buffer_distances)):
    resi_blks3['buffer_' + str(buffer_distances[i])] = resi_blks3['geometry'].buffer(buffer_distances[i])

resi_blks3.head()

,blk_no,street,Latitude,Longitude,Postcode,geometry,buffer_400,buffer_800,buffer_1200,buffer_1600,buffer_2000
0,1,BEACH RD,1.303671,103.864479,190001,POINT (373669.075 144123.588),"POLYGON ((374069.075 144123.588, 374067.149 14...","POLYGON ((374469.075 144123.588, 374465.223 14...","POLYGON ((374869.075 144123.588, 374863.297 14...","POLYGON ((375269.075 144123.588, 375261.371 14...","POLYGON ((375669.075 144123.588, 375659.445 14..."
1,1,BEDOK STH AVE 1,1.320852,103.933721,460001,POINT (381374.284 146019.548),"POLYGON ((381774.284 146019.548, 381772.357 14...","POLYGON ((382174.284 146019.548, 382170.431 14...","POLYGON ((382574.284 146019.548, 382568.505 14...","POLYGON ((382974.284 146019.548, 382966.579 14...","POLYGON ((383374.284 146019.548, 383364.653 14..."
2,1,CHAI CHEE RD,1.327969,103.922716,461001,POINT (380150.144 146806.830),"POLYGON ((380550.144 146806.830, 380548.218 14...","POLYGON ((380950.144 146806.830, 380946.292 14...","POLYGON ((381350.144 146806.830, 381344.366 14...","POLYGON ((381750.144 146806.830, 381742.440 14...","POLYGON ((382150.144 146806.830, 382140.514 14..."
3,1,CHANGI VILLAGE RD,1.388610,103.988093,500001,POINT (387427.101 153507.525),"POLYGON ((387827.101 153507.525, 387825.175 15...","POLYGON ((388227.101 153507.525, 388223.249 15...","POLYGON ((388627.101 153507.525, 388621.323 15...","POLYGON ((389027.101 153507.525, 389019.397 15...","POLYGON ((389427.101 153507.525, 389417.471 15..."
4,1,DELTA AVE,1.292075,103.828584,160001,POINT (369674.526 142843.424),"POLYGON ((370074.526 142843.424, 370072.600 14...","POLYGON ((370474.526 142843.424, 370470.674 14...","POLYGON ((370874.526 142843.424, 370868.748 14...","POLYGON ((371274.526 142843.424, 371266.822 14...","POLYGON ((371674.526 142843.424, 371664.896 14..."


### Check Using Folium

In [130]:
test_buffers = resi_blks3.loc[[0,1],:]

# Convert crs to epsg 4326 to match what Folium supports
test_buffers = test_buffers.to_crs("EPSG:4326")
for i in range(len(buffer_distances)):
    test_buffers = test_buffers.set_geometry('buffer_' + str(buffer_distances[i])).to_crs("EPSG:4326")

test_buffers

,blk_no,street,Latitude,Longitude,Postcode,geometry,buffer_400,buffer_800,buffer_1200,buffer_1600,buffer_2000
0,1,BEACH RD,1.303671,103.864479,190001,POINT (103.86448 1.30367),"POLYGON ((103.86807 1.30367, 103.86806 1.30332...","POLYGON ((103.87167 1.30367, 103.87163 1.30297...","POLYGON ((103.87526 1.30368, 103.87521 1.30261...","POLYGON ((103.87886 1.30368, 103.87879 1.30226...","POLYGON ((103.88245 1.30368, 103.88237 1.30191..."
1,1,BEDOK STH AVE 1,1.320852,103.933721,460001,POINT (103.93372 1.32085),"POLYGON ((103.93732 1.32085, 103.93730 1.32050...","POLYGON ((103.94091 1.32086, 103.94088 1.32015...","POLYGON ((103.94451 1.32086, 103.94445 1.31979...","POLYGON ((103.94810 1.32086, 103.94803 1.31944...","POLYGON ((103.95170 1.32086, 103.95161 1.31909..."


In [131]:
#basemap
map1 = folium.Map(location=([1.3521,103.8198]),zoom_start=12)

for _,row in test_buffers.iterrows():
    #add markers for blk locations
    row['location'] = (row['geometry'].x,row['geometry'].y)
    folium.map.Marker(location=list(reversed(row['location'])),
                     icon=folium.Icon(color='red',
                                     icon_color='black',
                                     icon='home'),
                     popup=row['blk_no']+" "+row['street']).add_to(map1)
    
    #add buffers
    for i in range(len(buffer_distances)):
        buffer_gs = gpd.GeoSeries(row['buffer_' + str(buffer_distances[i])])
        buffer_json = buffer_gs.to_json()
        folium.GeoJson(data=buffer_json).add_to(map1)

map1

In [35]:
del resi_blks3_checkbuffers,test_buffers,map1

## <font color = 'blue'> Shared Functions

**Function to count number of a specified amenity type within a specified buffer polygon**

In [36]:
def count_amenity(amenity_gdf, poly_buffer):
    """
    Compute number of amenities within the specified buffer polygon
    amenity_gdf = geodataframe for amenity type of interest
    polygon_buffer = buffer polygon to search within
    """
    
    return (len(amenity_gdf.loc[amenity_gdf.within(poly_buffer)]))

In [40]:
def dist_to_amenity(amenity_gdf, row):
    """
    Compute distance to nearest amenity
    amenity_gdf = geodataframe for amenity type of interest
    row = row of data with various buffer polygons to search within
    """
#     print('trying 400m')
    #list amenities within 400m
    mybuffer = row.buffer_400
    list_amen = amenity_gdf.loc[amenity_gdf.within(mybuffer)]
    
    #if none within 400m, widen buffer to 800m and so on
    if len(list_amen)==0:
#         print('trying 800m')
        mybuffer = row.buffer_800
        list_amen = amenity_gdf.loc[amenity_gdf.within(mybuffer)]
        
        if len(list_amen)==0:
#             print('trying 1200m')
            mybuffer = row.buffer_1200
            list_amen = amenity_gdf.loc[amenity_gdf.within(mybuffer)]
        
            if len(list_amen)==0:
#                 print('trying 1600m')
                mybuffer = row.buffer_1600
                list_amen = amenity_gdf.loc[amenity_gdf.within(mybuffer)]
                
                if len(list_amen)==0:
#                     print('trying 2000m')
                    mybuffer = row.buffer_2000
                    list_amen = amenity_gdf.loc[amenity_gdf.within(mybuffer)]
    
    #if no amenity within 2km, return null for distance
    if len(list_amen)==0:
        return(np.nan)
    else:
        list_amen['distance'] = list_amen['geometry'].distance(row.geometry)
        return(min(list_amen['distance']))

## <font color = 'blue'> Supermarkets

In [21]:
supmkt = gpd.read_file("Supermarkets/supermarkets shapefile/supermarkets.shp")
#set crs to match resi_blks3. should be EPSG:32648
supmkt = supmkt.to_crs(resi_blks3.crs)
supmkt.head(3)

,Lic_Name,Address,Postcode,geometry
0,LI LI CHENG SUPERMARKET (PUNGGOL) PTE. LTD.,"273C PUNGGOL PLACE, #884",823273,POINT Z (377766.852 155025.611 0.000)
1,SHENG SIONG SUPERMARKET PTE LTD,"11 UPPER BOON KENG ROAD, #901",380011,POINT Z (374385.654 145291.584 0.000)
2,COLD STORAGE SINGAPORE (1983) PTE LTD,"683 HOUGANG AVENUE 8, #903",530683,POINT Z (376107.879 151822.408 0.000)


### No. of Supermarkets Within 400m

In [37]:
amenities_1 = resi_blks3.copy()
amenities_1.head(3)

,blk_no,street,Latitude,Longitude,Postcode,geometry,buffer_400,buffer_800,buffer_1200,buffer_1600,buffer_2000
0,1,BEACH RD,1.303671,103.864479,190001,POINT (373669.075 144123.588),"POLYGON ((374069.075 144123.588, 374067.149 14...","POLYGON ((374469.075 144123.588, 374465.223 14...","POLYGON ((374869.075 144123.588, 374863.297 14...","POLYGON ((375269.075 144123.588, 375261.371 14...","POLYGON ((375669.075 144123.588, 375659.445 14..."
1,1,BEDOK STH AVE 1,1.320852,103.933721,460001,POINT (381374.284 146019.548),"POLYGON ((381774.284 146019.548, 381772.357 14...","POLYGON ((382174.284 146019.548, 382170.431 14...","POLYGON ((382574.284 146019.548, 382568.505 14...","POLYGON ((382974.284 146019.548, 382966.579 14...","POLYGON ((383374.284 146019.548, 383364.653 14..."
2,1,CHAI CHEE RD,1.327969,103.922716,461001,POINT (380150.144 146806.830),"POLYGON ((380550.144 146806.830, 380548.218 14...","POLYGON ((380950.144 146806.830, 380946.292 14...","POLYGON ((381350.144 146806.830, 381344.366 14...","POLYGON ((381750.144 146806.830, 381742.440 14...","POLYGON ((382150.144 146806.830, 382140.514 14..."


In [38]:
amenities_1['n_supmkt_400m'] = amenities_1['buffer_400'].progress_apply(lambda x: count_amenity(supmkt, x))

100%|██████████████████████████████████████████████████████████████████████████████| 9872/9872 [03:27<00:00, 47.62it/s]


In [41]:
amenities_1.head()

,blk_no,street,Latitude,Longitude,Postcode,geometry,buffer_400,buffer_800,buffer_1200,buffer_1600,buffer_2000,n_supmkt_400m
0,1,BEACH RD,1.303671,103.864479,190001,POINT (373669.075 144123.588),"POLYGON ((374069.075 144123.588, 374067.149 14...","POLYGON ((374469.075 144123.588, 374465.223 14...","POLYGON ((374869.075 144123.588, 374863.297 14...","POLYGON ((375269.075 144123.588, 375261.371 14...","POLYGON ((375669.075 144123.588, 375659.445 14...",1
1,1,BEDOK STH AVE 1,1.320852,103.933721,460001,POINT (381374.284 146019.548),"POLYGON ((381774.284 146019.548, 381772.357 14...","POLYGON ((382174.284 146019.548, 382170.431 14...","POLYGON ((382574.284 146019.548, 382568.505 14...","POLYGON ((382974.284 146019.548, 382966.579 14...","POLYGON ((383374.284 146019.548, 383364.653 14...",2
2,1,CHAI CHEE RD,1.327969,103.922716,461001,POINT (380150.144 146806.830),"POLYGON ((380550.144 146806.830, 380548.218 14...","POLYGON ((380950.144 146806.830, 380946.292 14...","POLYGON ((381350.144 146806.830, 381344.366 14...","POLYGON ((381750.144 146806.830, 381742.440 14...","POLYGON ((382150.144 146806.830, 382140.514 14...",1
3,1,CHANGI VILLAGE RD,1.388610,103.988093,500001,POINT (387427.101 153507.525),"POLYGON ((387827.101 153507.525, 387825.175 15...","POLYGON ((388227.101 153507.525, 388223.249 15...","POLYGON ((388627.101 153507.525, 388621.323 15...","POLYGON ((389027.101 153507.525, 389019.397 15...","POLYGON ((389427.101 153507.525, 389417.471 15...",0
4,1,DELTA AVE,1.292075,103.828584,160001,POINT (369674.526 142843.424),"POLYGON ((370074.526 142843.424, 370072.600 14...","POLYGON ((370474.526 142843.424, 370470.674 14...","POLYGON ((370874.526 142843.424, 370868.748 14...","POLYGON ((371274.526 142843.424, 371266.822 14...","POLYGON ((371674.526 142843.424, 371664.896 14...",3


### Distance to nearest supermarket

In [51]:
amenities_2 = amenities_1.copy()
amenities_2['dist_supmkt'] = amenities_2.progress_apply(lambda row: dist_to_amenity(supmkt, row),axis=1)

100%|██████████████████████████████████████████████████████████████████████████████| 9872/9872 [04:25<00:00, 37.23it/s]


In [52]:
amenities_2.head()

,blk_no,street,Latitude,Longitude,Postcode,geometry,buffer_400,buffer_800,buffer_1200,buffer_1600,buffer_2000,n_supmkt_400m,dist_supmkt
0,1,BEACH RD,1.303671,103.864479,190001,POINT (373669.075 144123.588),"POLYGON ((374069.075 144123.588, 374067.149 14...","POLYGON ((374469.075 144123.588, 374465.223 14...","POLYGON ((374869.075 144123.588, 374863.297 14...","POLYGON ((375269.075 144123.588, 375261.371 14...","POLYGON ((375669.075 144123.588, 375659.445 14...",1,127.108874
1,1,BEDOK STH AVE 1,1.320852,103.933721,460001,POINT (381374.284 146019.548),"POLYGON ((381774.284 146019.548, 381772.357 14...","POLYGON ((382174.284 146019.548, 382170.431 14...","POLYGON ((382574.284 146019.548, 382568.505 14...","POLYGON ((382974.284 146019.548, 382966.579 14...","POLYGON ((383374.284 146019.548, 383364.653 14...",2,115.486930
2,1,CHAI CHEE RD,1.327969,103.922716,461001,POINT (380150.144 146806.830),"POLYGON ((380550.144 146806.830, 380548.218 14...","POLYGON ((380950.144 146806.830, 380946.292 14...","POLYGON ((381350.144 146806.830, 381344.366 14...","POLYGON ((381750.144 146806.830, 381742.440 14...","POLYGON ((382150.144 146806.830, 382140.514 14...",1,399.600199
3,1,CHANGI VILLAGE RD,1.388610,103.988093,500001,POINT (387427.101 153507.525),"POLYGON ((387827.101 153507.525, 387825.175 15...","POLYGON ((388227.101 153507.525, 388223.249 15...","POLYGON ((388627.101 153507.525, 388621.323 15...","POLYGON ((389027.101 153507.525, 389019.397 15...","POLYGON ((389427.101 153507.525, 389417.471 15...",0,NaN
4,1,DELTA AVE,1.292075,103.828584,160001,POINT (369674.526 142843.424),"POLYGON ((370074.526 142843.424, 370072.600 14...","POLYGON ((370474.526 142843.424, 370470.674 14...","POLYGON ((370874.526 142843.424, 370868.748 14...","POLYGON ((371274.526 142843.424, 371266.822 14...","POLYGON ((371674.526 142843.424, 371664.896 14...",3,210.457681


In [57]:
#show percentage of blocks that do not have supmkt within 2km (to ensure this is an adequate max search radius)
n_null = len(amenities_2[amenities_2['dist_supmkt'].notnull()])
print(f"Percentage Blocks Covered = {round(n_null/len(amenities_2)*100,2)} %")

Percentage Blocks Covered = 99.96 %


## <font color = 'blue'> F&B Establishments

In [59]:
food = gpd.read_file("F&B Establishments/FOOD.shp")
food = food.to_crs(amenities_2.crs) #reproject
food.head()

,Postal cod,Cnt,Lat,Long,geometry
0,18906,1,1.275805,103.849615,POINT (372013.824 141043.590)
1,18935,50,1.277526,103.852935,POINT (372383.359 141233.758)
2,18936,2,1.278032,103.852883,POINT (372377.587 141289.715)
3,18937,1,1.277337,103.852382,POINT (372321.828 141212.839)
4,18946,1,1.270045,103.857324,POINT (372871.311 140406.456)


### No. of F&B Establishments within 400m

In [65]:
#need to use special function because there can be multiple food establishments at each location
def count_food(poly_buffer):
    """Compute number of food establishments within the specified buffer polygon"""
    return(food.loc[food.within(poly_buffer)]['Cnt'].sum())

In [66]:
amenities_3 = amenities_2.copy()
amenities_3['n_food_400m'] = amenities_3['buffer_400'].progress_apply(lambda x: count_food(x))

100%|██████████████████████████████████████████████████████████████████████████████| 9872/9872 [07:20<00:00, 22.42it/s]


In [67]:
amenities_3.head()

,blk_no,street,Latitude,Longitude,Postcode,geometry,buffer_400,buffer_800,buffer_1200,buffer_1600,buffer_2000,n_supmkt_400m,dist_supmkt,n_food_400m
0,1,BEACH RD,1.303671,103.864479,190001,POINT (373669.075 144123.588),"POLYGON ((374069.075 144123.588, 374067.149 14...","POLYGON ((374469.075 144123.588, 374465.223 14...","POLYGON ((374869.075 144123.588, 374863.297 14...","POLYGON ((375269.075 144123.588, 375261.371 14...","POLYGON ((375669.075 144123.588, 375659.445 14...",1,127.108874,155
1,1,BEDOK STH AVE 1,1.320852,103.933721,460001,POINT (381374.284 146019.548),"POLYGON ((381774.284 146019.548, 381772.357 14...","POLYGON ((382174.284 146019.548, 382170.431 14...","POLYGON ((382574.284 146019.548, 382568.505 14...","POLYGON ((382974.284 146019.548, 382966.579 14...","POLYGON ((383374.284 146019.548, 383364.653 14...",2,115.486930,31
2,1,CHAI CHEE RD,1.327969,103.922716,461001,POINT (380150.144 146806.830),"POLYGON ((380550.144 146806.830, 380548.218 14...","POLYGON ((380950.144 146806.830, 380946.292 14...","POLYGON ((381350.144 146806.830, 381344.366 14...","POLYGON ((381750.144 146806.830, 381742.440 14...","POLYGON ((382150.144 146806.830, 382140.514 14...",1,399.600199,5
3,1,CHANGI VILLAGE RD,1.388610,103.988093,500001,POINT (387427.101 153507.525),"POLYGON ((387827.101 153507.525, 387825.175 15...","POLYGON ((388227.101 153507.525, 388223.249 15...","POLYGON ((388627.101 153507.525, 388621.323 15...","POLYGON ((389027.101 153507.525, 389019.397 15...","POLYGON ((389427.101 153507.525, 389417.471 15...",0,NaN,54
4,1,DELTA AVE,1.292075,103.828584,160001,POINT (369674.526 142843.424),"POLYGON ((370074.526 142843.424, 370072.600 14...","POLYGON ((370474.526 142843.424, 370470.674 14...","POLYGON ((370874.526 142843.424, 370868.748 14...","POLYGON ((371274.526 142843.424, 371266.822 14...","POLYGON ((371674.526 142843.424, 371664.896 14...",3,210.457681,64


## <font color = 'blue'> Train Stations

In [68]:
mrt = gpd.read_file("MRT Station/MRTLRTStnPtt.shp")
mrt.head()

,OBJECTID,STN_NAME,STN_NO,geometry
0,1,EUNOS MRT STATION,EW7,POINT (35782.955 33560.078)
1,2,CHINESE GARDEN MRT STATION,EW25,POINT (16790.747 36056.302)
2,3,KHATIB MRT STATION,NS14,POINT (27962.311 44352.568)
3,4,KRANJI MRT STATION,NS7,POINT (20081.697 45214.548)
4,5,REDHILL MRT STATION,EW18,POINT (26163.478 30218.820)


In [70]:
#remove lrt stations
mrt['MRT'] = mrt['STN_NAME'].apply(lambda s: "MRT" in s)
mrt_1 = mrt[mrt['MRT']==True].reset_index(drop=True).drop(columns=['MRT'])
mrt_1.head()

,OBJECTID,STN_NAME,STN_NO,geometry
0,1,EUNOS MRT STATION,EW7,POINT (35782.955 33560.078)
1,2,CHINESE GARDEN MRT STATION,EW25,POINT (16790.747 36056.302)
2,3,KHATIB MRT STATION,NS14,POINT (27962.311 44352.568)
3,4,KRANJI MRT STATION,NS7,POINT (20081.697 45214.548)
4,5,REDHILL MRT STATION,EW18,POINT (26163.478 30218.820)


In [71]:
mrt_1 = mrt_1.to_crs(amenities_3.crs) #reproject
mrt_1.crs

<Projected CRS: EPSG:32648>
Name: WGS 84 / UTM zone 48N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 102°E and 108°E, northern hemisphere between equator and 84°N, onshore and offshore. Cambodia. China. Indonesia. Laos. Malaysia - West Malaysia. Mongolia. Russian Federation. Singapore. Thailand. Vietnam.
- bounds: (102.0, 0.0, 108.0, 84.0)
Coordinate Operation:
- name: UTM zone 48N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### No. of MRT Stations within 400m

In [72]:
amenities_4 = amenities_3.copy()
amenities_4['n_mrt_400m'] = amenities_4['buffer_400'].progress_apply(lambda x: count_amenity(mrt_1, x))

100%|██████████████████████████████████████████████████████████████████████████████| 9872/9872 [03:01<00:00, 54.28it/s]


In [73]:
amenities_4.head()

,blk_no,street,Latitude,Longitude,Postcode,geometry,buffer_400,buffer_800,buffer_1200,buffer_1600,buffer_2000,n_supmkt_400m,dist_supmkt,n_food_400m,n_mrt_400m
0,1,BEACH RD,1.303671,103.864479,190001,POINT (373669.075 144123.588),"POLYGON ((374069.075 144123.588, 374067.149 14...","POLYGON ((374469.075 144123.588, 374465.223 14...","POLYGON ((374869.075 144123.588, 374863.297 14...","POLYGON ((375269.075 144123.588, 375261.371 14...","POLYGON ((375669.075 144123.588, 375659.445 14...",1,127.108874,155,0
1,1,BEDOK STH AVE 1,1.320852,103.933721,460001,POINT (381374.284 146019.548),"POLYGON ((381774.284 146019.548, 381772.357 14...","POLYGON ((382174.284 146019.548, 382170.431 14...","POLYGON ((382574.284 146019.548, 382568.505 14...","POLYGON ((382974.284 146019.548, 382966.579 14...","POLYGON ((383374.284 146019.548, 383364.653 14...",2,115.486930,31,0
2,1,CHAI CHEE RD,1.327969,103.922716,461001,POINT (380150.144 146806.830),"POLYGON ((380550.144 146806.830, 380548.218 14...","POLYGON ((380950.144 146806.830, 380946.292 14...","POLYGON ((381350.144 146806.830, 381344.366 14...","POLYGON ((381750.144 146806.830, 381742.440 14...","POLYGON ((382150.144 146806.830, 382140.514 14...",1,399.600199,5,0
3,1,CHANGI VILLAGE RD,1.388610,103.988093,500001,POINT (387427.101 153507.525),"POLYGON ((387827.101 153507.525, 387825.175 15...","POLYGON ((388227.101 153507.525, 388223.249 15...","POLYGON ((388627.101 153507.525, 388621.323 15...","POLYGON ((389027.101 153507.525, 389019.397 15...","POLYGON ((389427.101 153507.525, 389417.471 15...",0,NaN,54,0
4,1,DELTA AVE,1.292075,103.828584,160001,POINT (369674.526 142843.424),"POLYGON ((370074.526 142843.424, 370072.600 14...","POLYGON ((370474.526 142843.424, 370470.674 14...","POLYGON ((370874.526 142843.424, 370868.748 14...","POLYGON ((371274.526 142843.424, 371266.822 14...","POLYGON ((371674.526 142843.424, 371664.896 14...",3,210.457681,64,0


### Distance to nearest MRT Station

In [74]:
amenities_5 = amenities_4.copy()
amenities_5['dist_mrt']=amenities_5.progress_apply(lambda row: dist_to_amenity(mrt_1, row), axis=1)

100%|██████████████████████████████████████████████████████████████████████████████| 9872/9872 [07:28<00:00, 22.03it/s]


In [75]:
amenities_5.head()

,blk_no,street,Latitude,Longitude,Postcode,geometry,buffer_400,buffer_800,buffer_1200,buffer_1600,buffer_2000,n_supmkt_400m,dist_supmkt,n_food_400m,n_mrt_400m,dist_mrt
0,1,BEACH RD,1.303671,103.864479,190001,POINT (373669.075 144123.588),"POLYGON ((374069.075 144123.588, 374067.149 14...","POLYGON ((374469.075 144123.588, 374465.223 14...","POLYGON ((374869.075 144123.588, 374863.297 14...","POLYGON ((375269.075 144123.588, 375261.371 14...","POLYGON ((375669.075 144123.588, 375659.445 14...",1,127.108874,155,0,441.695647
1,1,BEDOK STH AVE 1,1.320852,103.933721,460001,POINT (381374.284 146019.548),"POLYGON ((381774.284 146019.548, 381772.357 14...","POLYGON ((382174.284 146019.548, 382170.431 14...","POLYGON ((382574.284 146019.548, 382568.505 14...","POLYGON ((382974.284 146019.548, 382966.579 14...","POLYGON ((383374.284 146019.548, 383364.653 14...",2,115.486930,31,0,540.757663
2,1,CHAI CHEE RD,1.327969,103.922716,461001,POINT (380150.144 146806.830),"POLYGON ((380550.144 146806.830, 380548.218 14...","POLYGON ((380950.144 146806.830, 380946.292 14...","POLYGON ((381350.144 146806.830, 381344.366 14...","POLYGON ((381750.144 146806.830, 381742.440 14...","POLYGON ((382150.144 146806.830, 382140.514 14...",1,399.600199,5,0,915.733304
3,1,CHANGI VILLAGE RD,1.388610,103.988093,500001,POINT (387427.101 153507.525),"POLYGON ((387827.101 153507.525, 387825.175 15...","POLYGON ((388227.101 153507.525, 388223.249 15...","POLYGON ((388627.101 153507.525, 388621.323 15...","POLYGON ((389027.101 153507.525, 389019.397 15...","POLYGON ((389427.101 153507.525, 389417.471 15...",0,NaN,54,0,NaN
4,1,DELTA AVE,1.292075,103.828584,160001,POINT (369674.526 142843.424),"POLYGON ((370074.526 142843.424, 370072.600 14...","POLYGON ((370474.526 142843.424, 370470.674 14...","POLYGON ((370874.526 142843.424, 370868.748 14...","POLYGON ((371274.526 142843.424, 371266.822 14...","POLYGON ((371674.526 142843.424, 371664.896 14...",3,210.457681,64,0,673.165241


## <font color = 'blue'>Distance to Central Business District

In [77]:
plan_area = gpd.read_file("Planning Areas/planning area shape file/plan_areas.shp")
plan_area = plan_area.to_crs(amenities_5.crs) # reproject
plan_area.head()

,Plan_Area,Region,geometry
0,BUKIT MERAH,CENTRAL REGION,"POLYGON Z ((368430.686 143093.485 0.000, 36848..."
1,BUKIT PANJANG,WEST REGION,"POLYGON Z ((363656.421 153704.444 0.000, 36371..."
2,BUKIT TIMAH,CENTRAL REGION,"POLYGON Z ((366236.364 149042.002 0.000, 36628..."
3,CENTRAL WATER CATCHMENT,NORTH REGION,"POLYGON Z ((367143.376 156363.221 0.000, 36716..."
4,CHANGI,EAST REGION,"POLYGON Z ((387297.555 154539.272 0.000, 38737..."


In [78]:
# Extract only downtown core (i.e. CBD)
downtown = plan_area[plan_area['Plan_Area']=='DOWNTOWN CORE'].reset_index(drop=True)
downtown

,Plan_Area,Region,geometry
0,DOWNTOWN CORE,CENTRAL REGION,"POLYGON Z ((373899.795 144144.583 0.000, 37394..."


In [79]:
amenities_6 = amenities_5.copy()
amenities_6['dist_cbd'] = amenities_6['geometry'].distance(downtown.loc[0,'geometry'])

amenities_6.head()

,blk_no,street,Latitude,Longitude,Postcode,geometry,buffer_400,buffer_800,buffer_1200,buffer_1600,buffer_2000,n_supmkt_400m,dist_supmkt,n_food_400m,n_mrt_400m,dist_mrt,dist_cbd
0,1,BEACH RD,1.303671,103.864479,190001,POINT (373669.075 144123.588),"POLYGON ((374069.075 144123.588, 374067.149 14...","POLYGON ((374469.075 144123.588, 374465.223 14...","POLYGON ((374869.075 144123.588, 374863.297 14...","POLYGON ((375269.075 144123.588, 375261.371 14...","POLYGON ((375669.075 144123.588, 375659.445 14...",1,127.108874,155,0,441.695647,64.059681
1,1,BEDOK STH AVE 1,1.320852,103.933721,460001,POINT (381374.284 146019.548),"POLYGON ((381774.284 146019.548, 381772.357 14...","POLYGON ((382174.284 146019.548, 382170.431 14...","POLYGON ((382574.284 146019.548, 382568.505 14...","POLYGON ((382974.284 146019.548, 382966.579 14...","POLYGON ((383374.284 146019.548, 383364.653 14...",2,115.486930,31,0,540.757663,7451.474775
2,1,CHAI CHEE RD,1.327969,103.922716,461001,POINT (380150.144 146806.830),"POLYGON ((380550.144 146806.830, 380548.218 14...","POLYGON ((380950.144 146806.830, 380946.292 14...","POLYGON ((381350.144 146806.830, 381344.366 14...","POLYGON ((381750.144 146806.830, 381742.440 14...","POLYGON ((382150.144 146806.830, 382140.514 14...",1,399.600199,5,0,915.733304,6559.589365
3,1,CHANGI VILLAGE RD,1.388610,103.988093,500001,POINT (387427.101 153507.525),"POLYGON ((387827.101 153507.525, 387825.175 15...","POLYGON ((388227.101 153507.525, 388223.249 15...","POLYGON ((388627.101 153507.525, 388621.323 15...","POLYGON ((389027.101 153507.525, 389019.397 15...","POLYGON ((389427.101 153507.525, 389417.471 15...",0,NaN,54,0,NaN,16243.891728
4,1,DELTA AVE,1.292075,103.828584,160001,POINT (369674.526 142843.424),"POLYGON ((370074.526 142843.424, 370072.600 14...","POLYGON ((370474.526 142843.424, 370470.674 14...","POLYGON ((370874.526 142843.424, 370868.748 14...","POLYGON ((371274.526 142843.424, 371266.822 14...","POLYGON ((371674.526 142843.424, 371664.896 14...",3,210.457681,64,0,673.165241,2137.148412


## <font color = 'blue'> Preschools

In [80]:
presch = gpd.read_file("Preschools/preschools shapefile/preschools.shp")
presch = presch.to_crs(amenities_6.crs) #reproject
presch.head()

,Name,Address,Postcode,geometry
0,BRILLIANT TOTS PTE. LTD.,"610, JURONG WEST STREET 65, #01 - 534, S 640610",640610,POINT Z (355464.540 147963.619 0.000)
1,BUBBLESLAND PLAYHOUSE PTE LTD,"238, COMPASSVALE WALK, #01 - 542, S 540238",540238,POINT Z (377477.132 153714.286 0.000)
2,BUCKET HOUSE PRESCHOOL,"39, WOODLANDS CLOSE, #01 - 62, MEGA@WOODLANDS,...",737856,POINT Z (367260.249 158979.100 0.000)
3,BUMBLE BEE CHILD CARE CENTRE,"369, WOODLANDS AVENUE 1, #01 - 853, S 730369",730369,POINT Z (365102.321 158473.813 0.000)
4,BUSY BEES SINGAPORE PTE LTD,"327B, ANCHORVALE ROAD, #01 - 322, S 542327",542327,POINT Z (376361.355 154290.434 0.000)


### No. of Preschools Within 400m

In [82]:
amenities_7 = amenities_6.copy()

amenities_7['n_presch_400m'] = amenities_7['buffer_400'].progress_apply(lambda x: count_amenity(presch, x))

100%|██████████████████████████████████████████████████████████████████████████████| 9872/9872 [04:21<00:00, 37.78it/s]


In [83]:
amenities_7.head()

,blk_no,street,Latitude,Longitude,Postcode,geometry,buffer_400,buffer_800,buffer_1200,buffer_1600,buffer_2000,n_supmkt_400m,dist_supmkt,n_food_400m,n_mrt_400m,dist_mrt,dist_cbd,n_presch_400m
0,1,BEACH RD,1.303671,103.864479,190001,POINT (373669.075 144123.588),"POLYGON ((374069.075 144123.588, 374067.149 14...","POLYGON ((374469.075 144123.588, 374465.223 14...","POLYGON ((374869.075 144123.588, 374863.297 14...","POLYGON ((375269.075 144123.588, 375261.371 14...","POLYGON ((375669.075 144123.588, 375659.445 14...",1,127.108874,155,0,441.695647,64.059681,1
1,1,BEDOK STH AVE 1,1.320852,103.933721,460001,POINT (381374.284 146019.548),"POLYGON ((381774.284 146019.548, 381772.357 14...","POLYGON ((382174.284 146019.548, 382170.431 14...","POLYGON ((382574.284 146019.548, 382568.505 14...","POLYGON ((382974.284 146019.548, 382966.579 14...","POLYGON ((383374.284 146019.548, 383364.653 14...",2,115.486930,31,0,540.757663,7451.474775,4
2,1,CHAI CHEE RD,1.327969,103.922716,461001,POINT (380150.144 146806.830),"POLYGON ((380550.144 146806.830, 380548.218 14...","POLYGON ((380950.144 146806.830, 380946.292 14...","POLYGON ((381350.144 146806.830, 381344.366 14...","POLYGON ((381750.144 146806.830, 381742.440 14...","POLYGON ((382150.144 146806.830, 382140.514 14...",1,399.600199,5,0,915.733304,6559.589365,3
3,1,CHANGI VILLAGE RD,1.388610,103.988093,500001,POINT (387427.101 153507.525),"POLYGON ((387827.101 153507.525, 387825.175 15...","POLYGON ((388227.101 153507.525, 388223.249 15...","POLYGON ((388627.101 153507.525, 388621.323 15...","POLYGON ((389027.101 153507.525, 389019.397 15...","POLYGON ((389427.101 153507.525, 389417.471 15...",0,NaN,54,0,NaN,16243.891728,0
4,1,DELTA AVE,1.292075,103.828584,160001,POINT (369674.526 142843.424),"POLYGON ((370074.526 142843.424, 370072.600 14...","POLYGON ((370474.526 142843.424, 370470.674 14...","POLYGON ((370874.526 142843.424, 370868.748 14...","POLYGON ((371274.526 142843.424, 371266.822 14...","POLYGON ((371674.526 142843.424, 371664.896 14...",3,210.457681,64,0,673.165241,2137.148412,8


## <font color = 'blue'> Schools

In [84]:
schools = gpd.read_file("Schools/schools shapefile/schools.shp")
schools = schools.to_crs(amenities_7.crs) #reproject
schools.head()

,name,address,postcode,main_lvl,Latitude,Longitude,geometry
0,ADMIRALTY PRIMARY SCHOOL,11 WOODLANDS CIRCLE,738907,PRIMARY,1.442550,103.800214,POINT (366526.043 159480.601)
1,ADMIRALTY SECONDARY SCHOOL,31 WOODLANDS CRESCENT,737916,SECONDARY,1.445891,103.802398,POINT (366769.296 159849.843)
2,AHMAD IBRAHIM PRIMARY SCHOOL,10 YISHUN STREET 11,768643,PRIMARY,1.433153,103.832942,POINT (370167.033 158439.842)
3,AHMAD IBRAHIM SECONDARY SCHOOL,751 YISHUN AVENUE 7,768928,SECONDARY,1.436060,103.829719,POINT (369808.516 158761.410)
4,AI TONG SCHOOL,100 Bright Hill Drive,579646,PRIMARY,1.360583,103.833020,POINT (370171.721 150417.030)


### No. of Schools Within 2km
(Students residing within 2km of the school get priority to attend that school)

In [85]:
amenities_8 = amenities_7.copy()
amenities_8['n_sch_2km'] = amenities_8['buffer_2000'].progress_apply(lambda x: count_amenity(schools, x))

100%|██████████████████████████████████████████████████████████████████████████████| 9872/9872 [03:18<00:00, 49.80it/s]


In [86]:
amenities_8.head()

,blk_no,street,Latitude,Longitude,Postcode,geometry,buffer_400,buffer_800,buffer_1200,buffer_1600,buffer_2000,n_supmkt_400m,dist_supmkt,n_food_400m,n_mrt_400m,dist_mrt,dist_cbd,n_presch_400m,n_sch_2km
0,1,BEACH RD,1.303671,103.864479,190001,POINT (373669.075 144123.588),"POLYGON ((374069.075 144123.588, 374067.149 14...","POLYGON ((374469.075 144123.588, 374465.223 14...","POLYGON ((374869.075 144123.588, 374863.297 14...","POLYGON ((375269.075 144123.588, 375261.371 14...","POLYGON ((375669.075 144123.588, 375659.445 14...",1,127.108874,155,0,441.695647,64.059681,1,4
1,1,BEDOK STH AVE 1,1.320852,103.933721,460001,POINT (381374.284 146019.548),"POLYGON ((381774.284 146019.548, 381772.357 14...","POLYGON ((382174.284 146019.548, 382170.431 14...","POLYGON ((382574.284 146019.548, 382568.505 14...","POLYGON ((382974.284 146019.548, 382966.579 14...","POLYGON ((383374.284 146019.548, 383364.653 14...",2,115.486930,31,0,540.757663,7451.474775,4,18
2,1,CHAI CHEE RD,1.327969,103.922716,461001,POINT (380150.144 146806.830),"POLYGON ((380550.144 146806.830, 380548.218 14...","POLYGON ((380950.144 146806.830, 380946.292 14...","POLYGON ((381350.144 146806.830, 381344.366 14...","POLYGON ((381750.144 146806.830, 381742.440 14...","POLYGON ((382150.144 146806.830, 382140.514 14...",1,399.600199,5,0,915.733304,6559.589365,3,12
3,1,CHANGI VILLAGE RD,1.388610,103.988093,500001,POINT (387427.101 153507.525),"POLYGON ((387827.101 153507.525, 387825.175 15...","POLYGON ((388227.101 153507.525, 388223.249 15...","POLYGON ((388627.101 153507.525, 388621.323 15...","POLYGON ((389027.101 153507.525, 389019.397 15...","POLYGON ((389427.101 153507.525, 389417.471 15...",0,NaN,54,0,NaN,16243.891728,0,0
4,1,DELTA AVE,1.292075,103.828584,160001,POINT (369674.526 142843.424),"POLYGON ((370074.526 142843.424, 370072.600 14...","POLYGON ((370474.526 142843.424, 370470.674 14...","POLYGON ((370874.526 142843.424, 370868.748 14...","POLYGON ((371274.526 142843.424, 371266.822 14...","POLYGON ((371674.526 142843.424, 371664.896 14...",3,210.457681,64,0,673.165241,2137.148412,8,12


### Best Primary School Ranking Within 2km
Ranking is from 2020 ranking by salary.sg [link](https://www.salary.sg/2020/best-primary-schools-2020-by-popularity/)   

In [87]:
pri_ranks = pd.read_csv("Schools/primary school ranking.csv")
pri_ranks.head()

,Primary School Ranking
0,1. Rosyth School – 2.48 (3.5 + 29/20)
1,2. CHIJ St. Nicholas Girls’ School (Primary Se...
2,3. Pei Hwa Presbyterian Primary School – 2.3 (...
3,4. Ai Tong School – 2.26 (29/9 + 26/20)
4,5. Nan Hua Primary School – 2.2 (3.5 + 18/20)


In [88]:
pri_ranks1 = pri_ranks.copy()

#separate out ranking
pri_ranks1['rank'] = pri_ranks1['Primary School Ranking'].str.split(". ",n=1,expand=True)[0]
#change ranking to numeric
pri_ranks1['rank'] = pri_ranks1['rank'].apply(int)

#separate out school name
pri_ranks1['x'] = pri_ranks1['Primary School Ranking'].str.split(". ",n=1,expand=True)[1]
pri_ranks1.drop(columns=['Primary School Ranking'],inplace=True)
pri_ranks1['name'] = pri_ranks1['x'].str.split(" –",n=1,expand=True)[0]
pri_ranks1.drop(columns=['x'],inplace=True)

#change school to upper case
pri_ranks1['name'] = pri_ranks1['name'].str.upper()

#replace weird apostrophes
pri_ranks1['name'] = pri_ranks1['name'].str.replace(pat="’", repl="'", case=False, regex=False)

#remove " (PRIMARY SECTION)" as this is not present in the schools gdf.
pri_ranks1['name'] = pri_ranks1['name'].str.replace(pat=" (PRIMARY SECTION)", repl="", case=False, regex=False)

#rename SCGS as there is a mismatch with the schools gdf
pri_ranks1['name'] = pri_ranks1['name'].str.replace(pat="SINGAPORE CHINESE GIRLS' SCHOOL (PRIMARY)", repl="SINGAPORE CHINESE GIRLS' PRIMARY SCHOOL", case=False, regex=False)

In [89]:
pri_ranks1.head()

,rank,name
0,1,ROSYTH SCHOOL
1,2,CHIJ ST. NICHOLAS GIRLS' SCHOOL
2,3,PEI HWA PRESBYTERIAN PRIMARY SCHOOL
3,4,AI TONG SCHOOL
4,5,NAN HUA PRIMARY SCHOOL


In [90]:
#match with schools shapefile
pri_ranks2 = gpd.GeoDataFrame(pri_ranks1.merge(schools,how='left',on='name'), crs=schools.crs)
pri_ranks2.head()

,rank,name,address,postcode,main_lvl,Latitude,Longitude,geometry
0,1,ROSYTH SCHOOL,21 Serangoon North Avenue 4,555855,PRIMARY,1.372858,103.874772,POINT (374817.872 151771.843)
1,2,CHIJ ST. NICHOLAS GIRLS' SCHOOL,501 ANG MO KIO STREET 13,569405,MIXED LEVELS,1.373477,103.834253,POINT (370309.597 151842.382)
2,3,PEI HWA PRESBYTERIAN PRIMARY SCHOOL,7 PEI WAH AVENUE,597610,PRIMARY,1.338056,103.776106,POINT (363837.764 147929.663)
3,4,AI TONG SCHOOL,100 Bright Hill Drive,579646,PRIMARY,1.360583,103.833020,POINT (370171.721 150417.030)
4,5,NAN HUA PRIMARY SCHOOL,30 Jalan Lempeng,128806,PRIMARY,1.319202,103.761095,POINT (362166.392 145845.984)


In [91]:
pri_ranks2.isnull().values.any() #verify no nulls, fully matched

False

In [92]:
pri_ranks_final = pri_ranks2.loc[:,['name','geometry','rank']]
pri_ranks_final.head()

,name,geometry,rank
0,ROSYTH SCHOOL,POINT (374817.872 151771.843),1
1,CHIJ ST. NICHOLAS GIRLS' SCHOOL,POINT (370309.597 151842.382),2
2,PEI HWA PRESBYTERIAN PRIMARY SCHOOL,POINT (363837.764 147929.663),3
3,AI TONG SCHOOL,POINT (370171.721 150417.030),4
4,NAN HUA PRIMARY SCHOOL,POINT (362166.392 145845.984),5


In [93]:
pri_ranks_final.crs #ensure correct crs

<Projected CRS: EPSG:32648>
Name: WGS 84 / UTM zone 48N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 102°E and 108°E, northern hemisphere between equator and 84°N, onshore and offshore. Cambodia. China. Indonesia. Laos. Malaysia - West Malaysia. Mongolia. Russian Federation. Singapore. Thailand. Vietnam.
- bounds: (102.0, 0.0, 108.0, 84.0)
Coordinate Operation:
- name: UTM zone 48N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [94]:
def pri_ranking(poly_buffer):
    """
    Function to identify best ranking out of schools within a specified polygon
    poly_buffer = polygon to search within
    """
    #list schools within the buffer
    myschools=pri_ranks_final.loc[pri_ranks_final.within(poly_buffer)]
    
    if (len(myschools)>0): #at least 1 school
        #calculate and return the minimum ranking
        minrank = min(myschools['rank'])
    else: #if no schools return null
        minrank = np.nan
    
    return(minrank)

In [95]:
amenities_9 = amenities_8.copy()

amenities_9['prank_2km'] = amenities_9['buffer_2000'].progress_apply(lambda x: pri_ranking(x))

100%|██████████████████████████████████████████████████████████████████████████████| 9872/9872 [02:59<00:00, 54.95it/s]


In [96]:
amenities_9.head()

,blk_no,street,Latitude,Longitude,Postcode,geometry,buffer_400,buffer_800,buffer_1200,buffer_1600,buffer_2000,n_supmkt_400m,dist_supmkt,n_food_400m,n_mrt_400m,dist_mrt,dist_cbd,n_presch_400m,n_sch_2km,prank_2km
0,1,BEACH RD,1.303671,103.864479,190001,POINT (373669.075 144123.588),"POLYGON ((374069.075 144123.588, 374067.149 14...","POLYGON ((374469.075 144123.588, 374465.223 14...","POLYGON ((374869.075 144123.588, 374863.297 14...","POLYGON ((375269.075 144123.588, 375261.371 14...","POLYGON ((375669.075 144123.588, 375659.445 14...",1,127.108874,155,0,441.695647,64.059681,1,4,146.0
1,1,BEDOK STH AVE 1,1.320852,103.933721,460001,POINT (381374.284 146019.548),"POLYGON ((381774.284 146019.548, 381772.357 14...","POLYGON ((382174.284 146019.548, 382170.431 14...","POLYGON ((382574.284 146019.548, 382568.505 14...","POLYGON ((382974.284 146019.548, 382966.579 14...","POLYGON ((383374.284 146019.548, 383364.653 14...",2,115.486930,31,0,540.757663,7451.474775,4,18,6.0
2,1,CHAI CHEE RD,1.327969,103.922716,461001,POINT (380150.144 146806.830),"POLYGON ((380550.144 146806.830, 380548.218 14...","POLYGON ((380950.144 146806.830, 380946.292 14...","POLYGON ((381350.144 146806.830, 381344.366 14...","POLYGON ((381750.144 146806.830, 381742.440 14...","POLYGON ((382150.144 146806.830, 382140.514 14...",1,399.600199,5,0,915.733304,6559.589365,3,12,21.0
3,1,CHANGI VILLAGE RD,1.388610,103.988093,500001,POINT (387427.101 153507.525),"POLYGON ((387827.101 153507.525, 387825.175 15...","POLYGON ((388227.101 153507.525, 388223.249 15...","POLYGON ((388627.101 153507.525, 388621.323 15...","POLYGON ((389027.101 153507.525, 389019.397 15...","POLYGON ((389427.101 153507.525, 389417.471 15...",0,NaN,54,0,NaN,16243.891728,0,0,NaN
4,1,DELTA AVE,1.292075,103.828584,160001,POINT (369674.526 142843.424),"POLYGON ((370074.526 142843.424, 370072.600 14...","POLYGON ((370474.526 142843.424, 370470.674 14...","POLYGON ((370874.526 142843.424, 370868.748 14...","POLYGON ((371274.526 142843.424, 371266.822 14...","POLYGON ((371674.526 142843.424, 371664.896 14...",3,210.457681,64,0,673.165241,2137.148412,8,12,15.0


## <font color = 'blue'> Area of Nature & Parks Within 400m

In [103]:
greenery = gpd.read_file("Nature & Parks/greenery/greenery.shp")
greenery = greenery.to_crs(amenities_9.crs) # reproject
greenery.head()

,type,geometry
0,nature,"POLYGON Z ((363372.572 150163.892 0.000, 36337..."
1,nature,"MULTIPOLYGON Z (((363341.569 149692.505 0.000,..."
2,nature,"MULTIPOLYGON Z (((363927.266 150508.961 0.000,..."
3,nature,"POLYGON Z ((363305.152 149627.511 0.000, 36326..."
4,nature,"MULTIPOLYGON Z (((364518.832 149258.524 0.000,..."


In [104]:
#this function is mainly for testing purpose, to visualise the cropped areas
def green_area(poly_buffer): 
    """List green areas (parks/open space/Nature Reserve/ National Park/  Nature Area) within specified polygon buffer"""
    
    #convert poly_buffer to geodataframe
    polybuffer_gdf = gpd.GeoDataFrame({'geometry':[poly_buffer]},crs=greenery.crs)
    
    #find the intersection polygons between poly_buffer and green spaces
    green_spaces = gpd.overlay(polybuffer_gdf,greenery,how='intersection')
    
    return (green_spaces)

In [105]:
def total_green_area(poly_buffer): 
    """Sum up total green area (in ha) (parks/open space/Nature Reserve/ National Park/  Nature Area) within specified polygon buffer"""
    
    #convert poly_buffer to geodataframe
    polybuffer_gdf = gpd.GeoDataFrame({'geometry':[poly_buffer]},crs=greenery.crs)
    
    #find the intersection polygons between poly_buffer and green spaces
    green_spaces = gpd.overlay(polybuffer_gdf,greenery,how='intersection')
    
    #calculate area of each green space
    green_spaces['area'] = green_spaces['geometry'].area
    
    total_area = green_spaces['area'].sum()
    total_area = total_area/10000 #convert m2 to ha
    
    return (total_area)

**Test on sample block**

In [106]:
#select a random block - key in a random number up to 9800 for loc
tester = amenities_9.loc[[9320],:].reset_index(drop=True)
tester

,blk_no,street,Latitude,Longitude,Postcode,geometry,buffer_400,buffer_800,buffer_1200,buffer_1600,buffer_2000,n_supmkt_400m,dist_supmkt,n_food_400m,n_mrt_400m,dist_mrt,dist_cbd,n_presch_400m,n_sch_2km,prank_2km
0,86,TELOK BLANGAH HTS,1.276756,103.808525,100086,POINT (367441.777 141150.899),"POLYGON ((367841.777 141150.899, 367839.851 14...","POLYGON ((368241.777 141150.899, 368237.925 14...","POLYGON ((368641.777 141150.899, 368635.999 14...","POLYGON ((369041.777 141150.899, 369034.073 14...","POLYGON ((369441.777 141150.899, 369432.147 14...",1,375.777827,23,0,677.504179,3666.176375,7,6,15.0


In [107]:
green_spaces = green_area(tester['buffer_400'][0])
green_spaces.head()

,type,geometry
0,nature,"MULTIPOLYGON Z (((367293.925 141371.080 0.000,..."
1,nature,"MULTIPOLYGON Z (((367682.904 141242.149 0.000,..."
2,park_os,"MULTIPOLYGON Z (((367518.178 141294.002 0.000,..."
3,nature,"MULTIPOLYGON Z (((367183.710 141364.332 0.000,..."
4,nature,"POLYGON Z ((367840.844 141131.896 0.000, 36781..."


In [108]:
#reproject crs to suit folium
tester_1 = tester.to_crs('EPSG:4326')
for i in range(len(buffer_distances)):
    tester_1 = tester_1.set_geometry('buffer_' + str(buffer_distances[i])).to_crs("EPSG:4326")
    
green_spaces_1 = green_spaces.to_crs('EPSG:4326')

In [109]:
#basemap
map1 = folium.Map(location=([1.3521,103.8198]),zoom_start=12)

#add markers for blk locations
for _,row in tester_1.iterrows():
    row['location'] = (row['geometry'].x,row['geometry'].y)
    folium.map.Marker(location=list(reversed(row['location'])),
                     icon=folium.Icon(color='red',
                                     icon_color='black',
                                     icon='home'),
                     popup=row['blk_no']+" "+row['street']).add_to(map1)
    
    #add 400m buffer
    buffer_gs = gpd.GeoSeries(row['buffer_400'])
    buffer_json = buffer_gs.to_json()
    folium.GeoJson(data=buffer_json).add_to(map1)

green = lambda x: {'fillColor': 'green','color':'green'}
#add green spaces
for _,row in green_spaces_1.iterrows():
    buffer_gs = gpd.GeoSeries(row['geometry'])
    buffer_json = buffer_gs.to_json()
    folium.GeoJson(data=buffer_json,style_function=green).add_to(map1)

In [110]:
map1

In [111]:
total_green_area(tester['buffer_400'][0]) #verified by summing up areas from green_spaces dataframe

11.042231920221164

**Expand to All Blocks**

In [115]:
amenities_10 = amenities_9.copy()
amenities_10['grnarea_400'] = amenities_10['buffer_400'].progress_apply(lambda x: total_green_area(x))

100%|████████████████████████████████████████████████████████████████████████████| 9872/9872 [1:27:49<00:00,  1.87it/s]


In [116]:
amenities_10.head()

,blk_no,street,Latitude,Longitude,Postcode,geometry,buffer_400,buffer_800,buffer_1200,buffer_1600,...,n_supmkt_400m,dist_supmkt,n_food_400m,n_mrt_400m,dist_mrt,dist_cbd,n_presch_400m,n_sch_2km,prank_2km,grnarea_400
0,1,BEACH RD,1.303671,103.864479,190001,POINT (373669.075 144123.588),"POLYGON ((374069.075 144123.588, 374067.149 14...","POLYGON ((374469.075 144123.588, 374465.223 14...","POLYGON ((374869.075 144123.588, 374863.297 14...","POLYGON ((375269.075 144123.588, 375261.371 14...",...,1,127.108874,155,0,441.695647,64.059681,1,4,146.0,5.580603
1,1,BEDOK STH AVE 1,1.320852,103.933721,460001,POINT (381374.284 146019.548),"POLYGON ((381774.284 146019.548, 381772.357 14...","POLYGON ((382174.284 146019.548, 382170.431 14...","POLYGON ((382574.284 146019.548, 382568.505 14...","POLYGON ((382974.284 146019.548, 382966.579 14...",...,2,115.486930,31,0,540.757663,7451.474775,4,18,6.0,2.071161
2,1,CHAI CHEE RD,1.327969,103.922716,461001,POINT (380150.144 146806.830),"POLYGON ((380550.144 146806.830, 380548.218 14...","POLYGON ((380950.144 146806.830, 380946.292 14...","POLYGON ((381350.144 146806.830, 381344.366 14...","POLYGON ((381750.144 146806.830, 381742.440 14...",...,1,399.600199,5,0,915.733304,6559.589365,3,12,21.0,0.324354
3,1,CHANGI VILLAGE RD,1.388610,103.988093,500001,POINT (387427.101 153507.525),"POLYGON ((387827.101 153507.525, 387825.175 15...","POLYGON ((388227.101 153507.525, 388223.249 15...","POLYGON ((388627.101 153507.525, 388621.323 15...","POLYGON ((389027.101 153507.525, 389019.397 15...",...,0,NaN,54,0,NaN,16243.891728,0,0,NaN,4.586773
4,1,DELTA AVE,1.292075,103.828584,160001,POINT (369674.526 142843.424),"POLYGON ((370074.526 142843.424, 370072.600 14...","POLYGON ((370474.526 142843.424, 370470.674 14...","POLYGON ((370874.526 142843.424, 370868.748 14...","POLYGON ((371274.526 142843.424, 371266.822 14...",...,3,210.457681,64,0,673.165241,2137.148412,8,12,15.0,0.000000


In [133]:
amenities_10.columns

Index(['blk_no', 'street', 'Latitude', 'Longitude', 'Postcode', 'geometry',
       'buffer_400', 'buffer_800', 'buffer_1200', 'buffer_1600', 'buffer_2000',
       'n_supmkt_400m', 'dist_supmkt', 'n_food_400m', 'n_mrt_400m', 'dist_mrt',
       'dist_cbd', 'n_presch_400m', 'n_sch_2km', 'prank_2km', 'grnarea_400'],
      dtype='object')

In [142]:
amenities_10[['blk_no', 'street', 'Latitude', 'Longitude', 'Postcode', 'geometry',
       'buffer_400', 'buffer_800', 'buffer_1200', 'buffer_1600', 'buffer_2000',
       'n_supmkt_400m', 'grnarea_400', 'dist_mrt']].head()

,blk_no,street,Latitude,Longitude,Postcode,geometry,buffer_400,buffer_800,buffer_1200,buffer_1600,buffer_2000,n_supmkt_400m,grnarea_400,dist_mrt
0,1,BEACH RD,1.303671,103.864479,190001,POINT (373669.075 144123.588),"POLYGON ((374069.075 144123.588, 374067.149 14...","POLYGON ((374469.075 144123.588, 374465.223 14...","POLYGON ((374869.075 144123.588, 374863.297 14...","POLYGON ((375269.075 144123.588, 375261.371 14...","POLYGON ((375669.075 144123.588, 375659.445 14...",1,5.580603,441.695647
1,1,BEDOK STH AVE 1,1.320852,103.933721,460001,POINT (381374.284 146019.548),"POLYGON ((381774.284 146019.548, 381772.357 14...","POLYGON ((382174.284 146019.548, 382170.431 14...","POLYGON ((382574.284 146019.548, 382568.505 14...","POLYGON ((382974.284 146019.548, 382966.579 14...","POLYGON ((383374.284 146019.548, 383364.653 14...",2,2.071161,540.757663
2,1,CHAI CHEE RD,1.327969,103.922716,461001,POINT (380150.144 146806.830),"POLYGON ((380550.144 146806.830, 380548.218 14...","POLYGON ((380950.144 146806.830, 380946.292 14...","POLYGON ((381350.144 146806.830, 381344.366 14...","POLYGON ((381750.144 146806.830, 381742.440 14...","POLYGON ((382150.144 146806.830, 382140.514 14...",1,0.324354,915.733304
3,1,CHANGI VILLAGE RD,1.388610,103.988093,500001,POINT (387427.101 153507.525),"POLYGON ((387827.101 153507.525, 387825.175 15...","POLYGON ((388227.101 153507.525, 388223.249 15...","POLYGON ((388627.101 153507.525, 388621.323 15...","POLYGON ((389027.101 153507.525, 389019.397 15...","POLYGON ((389427.101 153507.525, 389417.471 15...",0,4.586773,NaN
4,1,DELTA AVE,1.292075,103.828584,160001,POINT (369674.526 142843.424),"POLYGON ((370074.526 142843.424, 370072.600 14...","POLYGON ((370474.526 142843.424, 370470.674 14...","POLYGON ((370874.526 142843.424, 370868.748 14...","POLYGON ((371274.526 142843.424, 371266.822 14...","POLYGON ((371674.526 142843.424, 371664.896 14...",3,0.000000,673.165241


### Save to Shapefile

In [117]:
amenities_final = amenities_10.drop(columns=['buffer_400','buffer_800','buffer_1200','buffer_1600','buffer_2000'])
amenities_final.head()

,blk_no,street,Latitude,Longitude,Postcode,geometry,n_supmkt_400m,dist_supmkt,n_food_400m,n_mrt_400m,dist_mrt,dist_cbd,n_presch_400m,n_sch_2km,prank_2km,grnarea_400
0,1,BEACH RD,1.303671,103.864479,190001,POINT (373669.075 144123.588),1,127.108874,155,0,441.695647,64.059681,1,4,146.0,5.580603
1,1,BEDOK STH AVE 1,1.320852,103.933721,460001,POINT (381374.284 146019.548),2,115.486930,31,0,540.757663,7451.474775,4,18,6.0,2.071161
2,1,CHAI CHEE RD,1.327969,103.922716,461001,POINT (380150.144 146806.830),1,399.600199,5,0,915.733304,6559.589365,3,12,21.0,0.324354
3,1,CHANGI VILLAGE RD,1.388610,103.988093,500001,POINT (387427.101 153507.525),0,NaN,54,0,NaN,16243.891728,0,0,NaN,4.586773
4,1,DELTA AVE,1.292075,103.828584,160001,POINT (369674.526 142843.424),3,210.457681,64,0,673.165241,2137.148412,8,12,15.0,0.000000


In [120]:
fp="Clean Datasets/dist_to_amenities"
print('Check Filepath To Avoid Accidental Overwrite')
confirm = input("Proceed to Save File? [Y/N]: ")
if confirm =='Y':
    amenities_final.to_file(fp)
    print('Saved!')
else:
    print('Not Saved.')

Check Filepath To Avoid Accidental Overwrite
Proceed to Save File? [Y/N]: Y


C:\Users\sharo\AppData\Local\Temp/ipykernel_14988/295253466.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  amenities_final.to_file(fp)


Saved!
